In [79]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

## 1) The Dataset

In [80]:
path = 'data/datasets-resized'

#Loading images from folder
import glob
import cv2
from skimage import io

types = ['*.jpg', '*.jpeg']

#Creating list X and Y
X_train = []
y_train = []

# Iterate the folder with glob
for fpath in glob.glob(path + '/*.jpg'): 
    # Opening images with OpenCV    
    # Alternatives: PIL, Matplotlib    
    im = cv2.imread(fpath, 0) 
    if 'glass' in fpath:
        label = 0 
    elif 'metal' in fpath:
        label = 1
    elif 'paper' in fpath:
        label = 2 
    elif 'plastic' in fpath:
        label = 3
    elif 'cardboard' in fpath:
        label = 4
        
    #Put image array to X    
    #Put label to y    
    X_train.append(im)
    y_train.append(label)
        
#Convert X and y to numpy array
x_train = np.array(X_train)
y_train = np.array(y_train)

C:\Users\sterr\AppData\Local\Temp\ipykernel_23100\1027397133.py:36: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train = np.array(X_train)


In [81]:
print(y_train)

[4 4 4 ... 3 3 3]


In [82]:
model = Sequential()
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

NameError: name 'Sequential' is not defined

## 2) Baselines

- Random guess accuracy?

- Human-Level Performance accuracy?

- Multilayer Perceptron accuracy?

## 3) Preprocessing

Explain all the preprocessing steps you did.

## 4) CNN architecture

## 5) CNN training

- Loss function

- Optimizer (with its hyper-parameters)

- Metrics

- Learning curves

## 6) Model performance

Metrics plots

## 7) Error analysis

Top error interpretation